# Project_MovieScore_Analysis

```
영화 평점 데이터를 이용하여 영화 추천 시스템을 작성하시오
*입/출력 예시
당신의 아이디를 입력하시오? 1
당신과 가장 비슷한 영화를 본 고객 아이디는 7번 입니다.
당신과 가장 비슷한 장르를 가지고 있는 고객 아이디는 10번 입니다
당신에게 추천하는 영화는 xxxxx 입니다.

*조건
- 비슷한 영화를 본 고객 아이디는 '공통적으로 본 영화'가 가장 많은 사람의 아이디입니다.
- 비슷한 장르를 가지고 있는 고객 아이디는 
1) 최소 2편 이상의 영화를 공통적으로 본 경우
2) 1)번 조건을 만족하는 모든 고객 중 가장 유사하게 평점을 부여한 고객으로 찾습니다.
3) 당신에게 추천하는(아직 본적 없는) 영화는 2)번 조건을 만족하는 고객이 가장 높게 평점을 준 영화입니다.
```

In [135]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

unames=['user_id','gender','age','occupation','zip']
rnames=['user_id','movie_id','ratings','timestamp']
mnames=['movie_id','title','genres']

upath = r"C:\Users\Administrator\movie_data\movielens\users.dat"
rpath = r"C:\Users\Administrator\movie_data\movielens\ratings.dat"
mpath = r"C:\Users\Administrator\movie_data\movielens\movies.dat"


users = pd.read_csv(upath,sep="::", header=None, names = unames)
ratings = pd.read_csv(rpath,sep="::", header=None, names = rnames)
movies = pd.read_csv(mpath,sep="::", header=None, names = mnames)

data = pd.merge(pd.merge(ratings,users),movies)
data

,user_id,movie_id,ratings,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,M,18,17,47901,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,M,35,14,30030,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,M,18,17,92886,White Boys (1999),Drama
1000207,5851,3607,5,957756608,F,18,20,55410,One Little Indian (1973),Comedy|Drama|Western


## problem 1 : 가장 비슷한 영화를 본 고객 아이디는 출력

In [440]:
your_id = int(input("아이디_넘버를 넣어주세요 : "))

def similarity(data, user_id):
    """
        해당 유저와 가장 유사한영화를 본 고객 아이디를 찾는 함수    
    """
    # 해당 유저가 본 영화의 리스트
    movie_list = list(data[data.user_id == user_id].movie_id)
    # 해당 유저가 본 영화들만 남긴다.
    user_watched_movie = data[data.movie_id.map(lambda x : x in movie_list)]
    # 본 영화의 개수를 세기위한 pivot table 만들기
    table = user_watched_movie.pivot_table('ratings', index='movie_id', columns='user_id')
    # 본 영화에 대해서 True, 아닌 영화는 False로 나오도록 설정
    table = (table == table)
    # 본인을 제외하고 점수 가장 높은 사람의 번호 출력
    return table.sum().sort_values(ascending=False).index[1]
similarity(data, your_id)
print("당신과 가장 비슷한 영화를 본 고객 아이디는 %d번 입니다." % similarity(data,your_id))

아이디_넘버를 넣어주세요 : 300
당신과 가장 비슷한 영화를 본 고객 아이디는 4169번 입니다.


## problem 2 : 가장 비슷한 장르를 가지고 있는 고객 아이디 출력

In [418]:
# 무비 장르가 어떤 것들이 있는지 확인하기
movie_genres = []
def movie_genres_check(string):
    global movie_genres
    for genre in string.split("|"):
        if genre not in movie_genres:
            movie_genres.append(genre)
    return 0
data.genres.map(movie_genres_check)
movie_genres

['Drama',
 'Animation',
 "Children's",
 'Musical',
 'Romance',
 'Comedy',
 'Action',
 'Adventure',
 'Fantasy',
 'Sci-Fi',
 'War',
 'Thriller',
 'Crime',
 'Mystery',
 'Western',
 'Horror',
 'Film-Noir',
 'Documentary']

In [422]:
# 필요한 데이터만 가져오기
df = data[['user_id','movie_id','ratings','genres']].copy()
# 영화 별 중복 장르 분리하기
df.genres = df.genres.map(lambda x : x.split("|"))
df.tail()

,user_id,movie_id,ratings,genres
1000204,5949,2198,5,[Documentary]
1000205,5675,2703,3,[Drama]
1000206,5780,2845,1,[Drama]
1000207,5851,3607,5,"[Comedy, Drama, Western]"
1000208,5938,2909,4,[Documentary]


In [423]:
#user_id 로 그룹화해서 해당 유저가 본 장르들을 종합
genres_by_user = df.groupby('user_id')['genres'].sum()
genres_by_user

user_id
1       [Drama, Animation, Children's, Musical, Musica...
2       [Drama, Drama, Comedy, Drama, Drama, Action, D...
3       [Animation, Children's, Comedy, Action, Advent...
4       [Children's, Drama, Fantasy, Sci-Fi, Action, C...
5       [Drama, Animation, Children's, Comedy, Adventu...
                              ...                        
6036    [Drama, Musical, Romance, Drama, Animation, Ch...
6037    [Drama, Comedy, Drama, Adventure, Children's, ...
6038    [Action, Adventure, Romance, Sci-Fi, War, Dram...
6039    [Animation, Children's, Musical, Musical, Roma...
6040    [Drama, Adventure, Children's, Drama, Musical,...
Name: genres, Length: 6040, dtype: object

In [424]:
# 유저별, 장르별 점수를 넣기 위한 데이터 프레임 형성
genre_board = pd.DataFrame(genres_by_user)
for genre in movie_genres:
    genre_board[genre] = 0
genre_board.head(2)

,genres,Drama,Animation,Children's,Musical,Romance,Comedy,Action,Adventure,Fantasy,Sci-Fi,War,Thriller,Crime,Mystery,Western,Horror,Film-Noir,Documentary
user_id,,,,,,,,,,,,,,,,,,,
1,"[Drama, Animation, Children's, Musical, Musica...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[Drama, Drama, Comedy, Drama, Drama, Action, D...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [425]:
user_num = len(genre_board) # 유저 수
for i in range(1,user_num+1): 
    for genre in genre_board.columns[1:]:
         #각 장르 점수표에 해당 유저가 본 장르의 수만큼의 점수를 부여
        genre_board.loc[i, genre] = genre_board.loc[i,'genres'].count(genre)
genre_board

,genres,Drama,Animation,Children's,Musical,Romance,Comedy,Action,Adventure,Fantasy,Sci-Fi,War,Thriller,Crime,Mystery,Western,Horror,Film-Noir,Documentary
user_id,,,,,,,,,,,,,,,,,,,
1,"[Drama, Animation, Children's, Musical, Musica...",21,18,20,14,6,14,5,5,3,3,2,3,2,0,0,0,0,0
2,"[Drama, Drama, Comedy, Drama, Drama, Action, D...",79,0,0,0,24,25,56,19,1,17,15,31,12,3,3,2,1,0
3,"[Animation, Children's, Comedy, Action, Advent...",8,3,3,1,5,30,23,25,2,6,2,5,0,1,6,3,0,0
4,"[Children's, Drama, Fantasy, Sci-Fi, Action, C...",6,0,1,0,2,0,19,6,2,9,3,4,1,0,2,3,0,0
5,"[Drama, Animation, Children's, Comedy, Adventu...",104,4,6,3,30,56,31,9,0,15,6,39,21,8,1,10,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,"[Drama, Musical, Romance, Drama, Animation, Ch...",372,34,54,31,122,261,154,83,19,169,42,147,53,34,14,74,17,11
6037,"[Drama, Comedy, Drama, Adventure, Children's, ...",98,1,6,4,22,59,28,9,4,39,16,68,18,13,4,9,9,1
6038,"[Action, Adventure, Romance, Sci-Fi, War, Dram...",9,3,1,0,6,12,2,1,0,1,4,0,0,0,0,2,0,0


In [426]:
# 필요없는 부분 제거
del genre_board['genres']

In [441]:
# 편차가 가장 적은 사람 찾기
def min_dff_score(data, user_id):
    user_score = data.loc[user_id]
    diff_square = (data - user_score)**2
    deviation = diff_square/(user_score+0.1) # 0인 경우를 제거하기 위해
    result = deviation.sum(axis=1).sort_values()
    return result.index[1]

print("당신과 가장 비슷한 영화를 본 고객 아이디는 %d번 입니다." % similarity(data,min_dff_score(genre_board, your_id)))

당신과 가장 비슷한 영화를 본 고객 아이디는 4169번 입니다.
